### Frequency of ICD codes among patients with asthma-like symptoms 

In [ ]:
import urllib
import pprint, json, requests, mysql.connector
from greentranslator.api import GreenTranslator
import mysql.connector

import matplotlib
matplotlib.use('Agg')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


flatten = lambda l: [item for sublist in l for item in sublist]

try:
    cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='umls.ncats.io')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

#try:
#    conn=psycopg2.connect("host='aact-prod.cr4nrslb1lw7.us-east-1.rds.amazonaws.com:5432' dbname='aact' user='aact' password='aact'")
#except:
#    print("I am unable to connect to the database.")
#print(conn)
#cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
#cur = conn.cursor()

print("Done")

In [2]:
# Find UNC patients
def findPatients_unc(age='8', sex='male', race='white', location='OUTPATIENT'):
    query = GreenTranslator ().get_query()
    return query.clinical_get_patients(age, sex, race, location)

patients = findPatients_unc();

print("Number of patients %d"%(len(patients)))

pprint.pprint(patients)

Initializing thread provenance @thread: MainThread
Number of patients 25
[{'birth_date': '2009-04-02 00:00:00',
  'diag': {'ICD10:3E0G76Z': {'2017-02-07 00:00:00': 'OUTPATIENT'},
           'ICD10:A49.02': {'2017-02-11 00:00:00': 'OUTPATIENT'},
           'ICD10:B95.62': {'2016-12-25 00:00:00': 'OUTPATIENT'},
           'ICD10:B96.5': {'2017-01-17 00:00:00': 'OUTPATIENT'},
           'ICD10:D64.9': {'2016-04-28 00:00:00': 'OUTPATIENT'},
           'ICD10:D80.1': {'2016-05-01 00:00:00': 'OUTPATIENT'},
           'ICD10:E07.9': {'2017-01-17 00:00:00': 'OUTPATIENT'},
           'ICD10:E41': {'2017-02-06 00:00:00': 'OUTPATIENT'},
           'ICD10:E44.0': {'2016-12-25 00:00:00': 'OUTPATIENT'},
           'ICD10:E84.0': {'2017-02-12 00:00:00': 'OUTPATIENT'},
           'ICD10:E84.19': {'2016-12-25 00:00:00': 'OUTPATIENT'},
           'ICD10:E84.8': {'2016-12-25 00:00:00': 'OUTPATIENT'},
           'ICD10:E84.9': {'2017-02-05 00:00:00': 'OUTPATIENT'},
           'ICD10:J15.1': {'2017-02-12 0

In [3]:
## Given ICD 9 or ICD 10 code, return disease/condition term
def findICD_ohdsi(txt, icd_version = 9):
    if icd_version == 9:
        icd_type = 'ICD9CM'
    elif icd_version == 10:
        icd_type = 'ICD10'
    else: raise Exception("Invalid ICD version specified")    
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": txt,
              "VOCABULARY_ID": [icd_type]}
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    return [d["CONCEPT_CODE"] for d in data]
#print findICD_ohdsi('asthma')

# Get ICD10/ICD9 code for a given string from UMLS. By default we get back ICD10.
def decode_icd(name, icd_version = 10):
    #if icd_version == 9:
    #    icd_type = 'ICD9CM'
    #elif icd_version == 10:
    #    icd_type = 'ICD10'
    #else: raise Exception("Invalid ICD version specified")

    cursor = cnx.cursor()
    query = ("SELECT STR FROM umls.MRCONSO WHERE CODE='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    #if res != "Undef":
    #    query = ("SELECT STR FROM umls.MRCONSO WHERE SAB='"+icd_type+"' AND CUI='"+res[0]+"'")
    #    cursor.execute(query, ())
    #    icd10 = "Undef"
    #    for code in cursor:
    #        icd10 = code
    #    return (icd10[0])
    return (res)

#print(findICD_umls('Asthma'))
#print(findICD_umls('Asthma', 9))

In [4]:
# count up frequencies of diag codes
counts = {}

for p in patients:
    for d in p['diag'].keys():
        try:
            counts[d] = counts[d]+1
        except KeyError:
            counts[d] = 1
            
l = []
for c in counts:
    l.append((c,counts[c]))

s = sorted(l,key=lambda x: 100-x[1])

full_data = []

for q in s:
    x = q[0].split(":")
    if (len(x) > 1):
        name = decode_icd(x[1])
        full_data.append((q[0],q[1],name[0]))
        print("ID %s (%s) - %d"%(q[0],name[0],q[1]))
    else:
        name = ""
        print("ID %s - %d"%(q[0],q[1]))



ID ICD9:786.2 (Cough) - 18
ID ICD10:Z23 (Loss/death parent/family) - 17
ID ICD9:V04.81 (Vaccin for influenza) - 15
ID ICD9:V20.2 (Motorcycle rider injured in collision with pedestrian or animal, unspecified motorcycle rider, nontraffic accident) - 14
ID ICD9:465.9 (Acute uri NOS) - 14
ID ICD9:493.90 (Asthma NOS) - 13
ID ICD9:V58.69 (Encounter for long-term current use of other high-risk medications) - 12
ID ICD9:780.60 (Chills with fever) - 12
ID ICD9:477.9 (Allergic rhinitis NOS) - 11
ID ICD10:Z00.129 (Encounter for routine child health examination NOS) - 11
ID ICD9:786.07 (Wheezing) - 11
ID ICD10:Z79.899 (Other long term (current) drug therapy) - 11
ID ICD9:382.9 (Otitis media NOS) - 10
ID ICD10:R05 (Cough) - 9
ID ICD9:462 (LYMPHOMA) - 8
ID ICD10:J02.9 (Acute pharyngitis, unspecified) - 8
ID ICD10:K59.00 (Constipation, unspecified) - 8
ID ICD10:R50.9 (Fever, unspecified) - 8
ID ICD10:R10.9 (Unspecified abdominal pain) - 8
ID ICD9:478.19 (Abscess of nose) - 7
ID ICD9:079.99 (Viral inf

In [6]:
#print p_agelocspm25
def filter1(x):
    (_,_,z) = x
    return z

def filter2(x):
    (_,z,_) = x
    return z

cols = list(map(filter1,full_data))
data = list(map(filter2,full_data))

#print(data)
#print(cols)

d = pd.DataFrame({"count":data,"diag":cols}, columns=["count","diag"])

#print(d)

g = sns.barplot(x="count", y="diag", data=d, label="count")
#g.set_xticklabels(rotation=50)

g